In [15]:
!pip install transformers datasets torch PyPdf2 pdfplumber evaluate pyrouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191604 sha256=6777c1c833c73f2f7f3eca30990c879269dbb6fcfc1a216f87c4a280df6fd0d9
  Stored in directory: /root/.cache/pip/wheels/9a/67/12/c5dd8ef8b4152bb8789eafd2a74a734e2dc7bb9eae02b768e7
Successfully built pyrouge


In [2]:
from datasets import load_dataset

df = load_dataset("cnn_dailymail","1.0.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
df['train'][1]['article']


'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [ ]:
df['train'][1]['highlights']

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [4]:
#tokenization

def preprocess_function(batch):
    source = batch['article']
    target = batch["highlights"]
    source_ids = tokenizer(source, truncation=True, padding="max_length", max_length=1024)
    target_ids = tokenizer(target, truncation=True, padding="max_length", max_length=128)

    # Replace pad token id with -100 for labels to ignore padding in loss computation
    labels = target_ids["input_ids"]
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]

    return {
        "input_ids": source_ids["input_ids"],
        "attention_mask": source_ids["attention_mask"],
        "labels": labels
    }

df_source = df.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Define training arguments
training_args = TrainingArguments(
    output_dir="./content",  # Replace with your output directory
    per_device_train_batch_size=8,
    num_train_epochs=2,  # Adjust number of epochs as needed
    remove_unused_columns=False,
    fp16=True  # Enable mixed-precision training
)

# Create Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
  train_dataset = df_source["train"].select(range(30000)),  # Use first 20,000 examples
eval_dataset = df_source["test"].select(range(4000))  # Use first 4,000 examples
)

# Start training
trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,2.069700
1000,1.973200
1500,1.933400
2000,1.871300
2500,1.841000
3000,1.818000
3500,1.796400
4000,1.622100
4500,1.466500
5000,1.443200


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=7500, training_loss=1.6630483154296876, metrics={'train_runtime': 8841.4619, 'train_samples_per_second': 6.786, 'train_steps_per_second': 0.848, 'total_flos': 1.3002627612672e+17, 'train_loss': 1.6630483154296876, 'epoch': 2.0})

In [21]:
from tqdm import tqdm

# Generate summaries for the test set
def generate_predictions(dataset, model, tokenizer, device):
    predictions = []
    references = []
    for data in tqdm(dataset):
        # Prepare input
        input_ids = tokenizer.encode(
            data["article"],
            max_length=1024,
            return_tensors='pt',
            truncation=True
        ).to(device)

        # Generate summary
        summary_ids = model.generate(
            input_ids,
            num_beams=4,
            max_length=128,
            min_length=30,
            early_stopping=True
        )
        predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Store the prediction and reference
        predictions.append(predicted_summary)
        references.append(data["highlights"])

    return predictions, references

# Get predictions and references
test_dataset = df_source["test"].select(range(100))  # Use a subset for testing
predictions, references = generate_predictions(test_dataset, model, tokenizer, device)


100%|██████████| 100/100 [01:51<00:00,  1.11s/it]


In [22]:
from rouge_score import rouge_scorer

def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        scores["rouge1"].append(score["rouge1"].fmeasure)
        scores["rouge2"].append(score["rouge2"].fmeasure)
        scores["rougeL"].append(score["rougeL"].fmeasure)

    # Compute average scores
    avg_scores = {key: sum(value) / len(value) for key, value in scores.items()}
    return avg_scores

# Calculate and print ROUGE scores
rouge_scores = calculate_rouge(predictions, references)
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge1': 0.34662548088688, 'rouge2': 0.14573101410106498, 'rougeL': 0.2511451543100491}


In [9]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(eval_results)

{'eval_loss': 1.8181010484695435, 'eval_runtime': 143.9564, 'eval_samples_per_second': 27.786, 'eval_steps_per_second': 3.473, 'epoch': 2.0}


In [10]:
# Save the model and tokenizer after training
model.save_pretrained("/content/drive/MyDrive/Colab/model_directory")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab/model_directory")

('/content/drive/MyDrive/Colab/model_directory/tokenizer_config.json',
 '/content/drive/MyDrive/Colab/model_directory/special_tokens_map.json',
 '/content/drive/MyDrive/Colab/model_directory/vocab.json',
 '/content/drive/MyDrive/Colab/model_directory/merges.txt',
 '/content/drive/MyDrive/Colab/model_directory/added_tokens.json',
 '/content/drive/MyDrive/Colab/model_directory/tokenizer.json')

In [11]:
!ls /content/drive/MyDrive/Colab/model_directory

config.json		merges.txt	   special_tokens_map.json  tokenizer.json
generation_config.json	model.safetensors  tokenizer_config.json    vocab.json


In [13]:
import os
from google.colab import files
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from PyPDF2 import PdfReader
import torch
import re

# Load the trained model and tokenizer (same as before)

# Function to clean text
def clean_text(text):
  # Remove non-ASCII characters
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)
  # Replace multiple spaces or newlines with a single space
  text = re.sub(r'\s+', ' ', text)
  # Remove URLs (links)
  text = re.sub(r'https?://\S+|www\.\S+', '', text)
  # Remove symbols (except common punctuation)
  text = re.sub(r'[^\w\s\!\?\.\,\'\:\-]', '', text)
  return text.strip()

# Function to Calculate summary lengths based on text length (same as before)

# Function to summarize text
def summarize_text(pdf_file, summary_length):
  try:
    text = ""
    reader = PdfReader(pdf_file)
    for page in reader.pages:
      text += page.extract_text()
    text = clean_text(text)  # Apply cleaning before processing
    text_length = len(text.split())

    summary_range = calc_summary_lengths(text_length)
    if summary_length not in summary_range:
      return "Invalid summary length. Choose 'Short', 'Medium', or 'Long'."

    min_length, max_length = summary_range[summary_length]

    inputs = tokenizer.encode(
        text,
        max_length=1024,
        truncation=True,
        return_tensors='pt'
    ).to(device)
    summary_ids = model.generate(
        inputs,
        num_beams=4,
        min_length=min_length,
        max_length=max_length,
        length_penalty=1.5,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  except Exception as e:
    return f"Error: {str(e)}\nPlease check file size and type!"

# Upload PDF file from the local system
uploaded = files.upload()

# Get the file path of the uploaded PDF (same as before)

# Enter the desired summary length (Short, Medium, Long)
summary_length = input("Enter summary length (Short, Medium, Long): ").strip().capitalize()

# Process and summarize the PDF
if pdf_path:
  # Open the uploaded file as a readable object and pass it to the function
  with open(pdf_path, 'rb') as file:
    summary = summarize_text(file, summary_length)
    print("\nSummary:")
    # Make sure the summary is in proper English (consider additional steps if needed)
    summary = summary.title()  # Capitalize the first letter of each sentence
    print(summary.replace("  ", " "))  # Remove extra spaces
else:
  print("No file uploaded.")

Saving TalhaAbdulRehmanResume.pdf to TalhaAbdulRehmanResume (1).pdf
Enter summary length (Short, Medium, Long): long

Summary:
Talha Abdul Rehman Is A Computer Science Student From Rawalpindi, Pakistan . He Is Skilled In Web De Velopment, Game Creation With Unity And Lm Making . He Has E Xperience Building Responsive Websites Using Html, Css, And Javascript . His Ai Pr Ojects Demonstr Ate My Ability To Inno Vate, While His Lm M Making Skills Enhance His Storytelling And Presentation Capabilities. He Is Eager To Apply His Diverse Skill Set In A Professional Envir Onment And Contribute To Your Team .Learn More About Talha At Tarehman1Gmail.Com And Twitter.Com/Intalha-Abdul-R Ehman-824722260 Alha3002  Linkin.Com Www.Enhancv.Com.Org/Professionals/Jobs/Tales/Talha-Rehman-Rehan-1@Gmail.Co.Com, .
